In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline

data = pd.read_csv('example.csv', ';',encoding='latin1')
# data = pd.read_csv('~/Dropbox/workspace-python/LZDataPrepAndML/outV5.csv', '\t',encoding='latin1')
# data.count()


   Effects  Responses  steps   c1   c2   c3   c4   c5   c6   c7   c8   c9  \
0        4          3    0.2  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
1        4          3    0.2  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
2        4          3    0.2  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
3        4          3    0.2  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
4        4          3    0.2  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   

   c10  c11  c12  
0  0.0  0.0  1.0  
1  0.0  0.2  0.8  
2  0.0  0.4  0.6  
3  0.0  0.6  0.4  
4  0.0  0.8  0.2  


count    175616.0
mean          3.0
std           0.0
min           3.0
25%           3.0
50%           3.0
75%           3.0
max           3.0
Name: Responses, dtype: float64

In [2]:
print(data.head())

   EID  CID                Time Event                  Response
0    1    1   12-05-2017 09:53     va                   Warning
1    2    1   13-05-2017 13:35     po  DistractClient;Seclusion
2    3    1   26-05-2017 09:32     va                   Warning
3    4    1   26-05-2017 11:02     pp            DistractClient
4    5    2   21-06-2017 14:51     va            DistractClient


In [38]:
columnTime = "Time"
columnAction = "Event"
columnResponse = "Response"
columnCaseid = "CID"
columnEventid = "EID"

# columnTime = "Algemeen: Datum incident"
# columnAction = "Aggression_short"
# columnResponse = "[B09] Maatregelen om agressie te stoppen"
# columnCaseid = "Pseudoniem_client"
# columnEventid = "Meldingsnummer"



columnEffect = "Next_Type_Eps"
data[columnTime].describe()

count                      10
unique                     10
top       2017-06-24 17:02:00
freq                        1
first     2017-05-13 13:35:00
last      2017-12-05 09:53:00
Name: Time, dtype: object

In [4]:


data[columnTime] = pd.to_datetime(data[columnTime])
# data = data.sort_values('Algemeen: Datum incident') # This now sorts in date 

##
## Sorting the incident. Only then the shift works well. 
##
data.sort_values([columnCaseid, columnTime, columnEventid], ascending=[True, True, True], inplace=True)


In [5]:
#
# Shift the data that is grouped by the clients to calculate the next incident type and next incident date
#
data['Next_Type'] = data.groupby(columnCaseid)[columnAction].shift(-1)
data['Next_DayOfIncident'] = data.groupby(columnCaseid)[columnTime].shift(-1)

#
# Set the data type of the next date 
#
data['Next_DayOfIncident'] = pd.to_datetime(data['Next_DayOfIncident'])

#
# Calculate the difference. 
#
data["Next_DaysToNext"] = (data['Next_DayOfIncident'] - data[columnTime])/np.timedelta64(1,'D')

In [6]:
### Step 1 : preprocess log using epsilon
eps = 7

data.loc[data['Next_DaysToNext'] <= eps, columnEffect] = data['Next_Type']
data.loc[data['Next_DaysToNext'] > eps, columnEffect] = 'Tau' 

# data.fillna("NaN", inplace=True)

In [7]:

data[columnEffect].describe()

count       7
unique      4
top       Tau
freq        3
Name: Next_Type_Eps, dtype: object

In [8]:
cNextT = columnAction
cCurrT = columnEffect

setActions = pd.Series(data[columnAction]).drop_duplicates().tolist()
print(setActions)

columnname = columnResponse
cMeasureValues = pd.Series(data[columnResponse]).drop_duplicates().tolist()
# print(cMeasureValues)


setResponse = set()
for i in cMeasureValues:
    for v in str(i).split(';'):
        setResponse.add(v.strip())

print(setResponse)


['po', 'va', 'pp']
{'DistractClient', 'Warning', 'NoResponse', 'Seclusion'}


In [13]:

data['Next_Type_Eps'].replace(np.nan, "NaN", inplace = True)


# FF = calcFreqMeasuresCombined2DDict(data, columnname+cMeasureValues[0])
FF = calcFreqMeasures2DDict(data, columnAction, columnResponse, columnEffect, setResponse)
# F = calcFreq2DDict(data, cMeasureT)




label = "va"
F = FF[label]
print(F)

# for ai in F:
#     F[ai][ai] = 0


{'columnTot': {'pp': 1, 'Tau': 1, 'NaN': 2, 'va': 1, 'po': 1}, 'Warning': {'pp': 1, 'NaN': 1, 'po': 1}, 'DistractClient': {'Tau': 1, 'va': 1}, 'NoResponse': {'NaN': 1}}


In [18]:
# -----------------
# Example : two dimentional dictionary to data frame
# ------------------
# tableRe = pd.DataFrame(Frelative).T

# print(tableRe)
# chi2_cont_row(tableRe)

table = pd.DataFrame(F).T
print(table)
table.fillna(0, inplace=True)
table.drop(columns=['NaN'], inplace = True)
table.drop(index=['columnTot'], inplace = True)
table.dropna
print(table)
stat, p, dof, expected = chi2_cont_row(table)

                NaN  Tau   po   pp   va
columnTot       2.0  1.0  1.0  1.0  1.0
Warning         1.0  NaN  1.0  1.0  NaN
DistractClient  NaN  1.0  NaN  NaN  1.0
NoResponse      1.0  NaN  NaN  NaN  NaN
                Tau   po   pp   va
Warning         0.0  1.0  1.0  0.0
DistractClient  1.0  0.0  0.0  1.0
NoResponse      0.0  0.0  0.0  0.0


ValueError: The internally computed table of expected frequencies has a zero element at (2, 0).

In [15]:
import graphviz
import pygraphviz as pgv

# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm       

from itertools import combinations 

pos = "all"

label = 'va'
start = label
name = label
F = FF[label]
print(F)

table = pd.DataFrame(F).T
table.replace(np.nan,0, inplace = True)
table.drop(columns=['NaN'], inplace = True)
print(table)

G = pgv.AGraph(strict=False, directed=True, landscape=True)
G.graph_attr['rankdir'] ='LR'
G.node_attr['shape']= 'box'
G.clear()
    

pvalues = list() # keep a list of computed p values to use for adjusting p-values

adjresidual = 2.57
for ai in F:
    if(ai == 'columnTot'):
        continue
    print("=======")
    print(ai)
    newF = dict()
    newF['non'] = dict()
    newF[ai] = dict()
    bool_observed_zero = False
    for aj in table.columns:
        # expected cell >= 5
        # observed count cell >= 1
        # observed count 75% cells  >= 5

        if(aj not in F[ai] or F[ai][aj] == 0 or pd.isnull(F[ai][aj])):
            bool_observed_zero = True
            print(ai)
            break
    
        newF[ai][aj] = F[ai][aj]
        newF['non'][aj] = table.at['columnTot',aj] - F[ai][aj]
    if bool_observed_zero:
        print("skip")
        continue
    

    newTable = pd.DataFrame(newF).T
    stat, p, dof, expected = chi2_cont_row(newTable)
    
    expectedT = pd.DataFrame(expected, columns = newTable.columns, index = newTable.index)

    
    if (p <= 0.05):
        rtable = sm.stats.Table(newTable)     
#         print(rtable.resid_pearson)
#         print(rtable.standardized_resids)
        
        subtable = pd.DataFrame(rtable.standardized_resids)
#         print(subtable)
#         print(newTable)
#         print(expectedT)
        
        for aj in table.columns:
            if (subtable.at[ai,aj] < -adjresidual) or (subtable.at[ai,aj] > adjresidual):
                if(G.has_edge(start+"_s", ai) == False):
                    rowtot = newTable.sum(axis=1)[ai]
                    G.add_edge(start+"_s", ai, label="{0:.0f}".format(rowtot))
                if(subtable.at[ai,aj] > 2.57):
                    penwidthl='3' # TODO replace with a function 
                else:
                    penwidthl='1'
                G.add_edge(ai, aj, penwidth=penwidthl, label="{0:.0f}".format(newTable.at[ai,aj]) + "("+ "{0:.0f}".format(expectedT.at[ai,aj]) +")")
    pvalues.append(p)
    
G.draw('./graph_' + name + '-asr' + pos + '.png', prog='dot')
    
    
p_adjusted = multipletests(pvalues, alpha=.05, method='bonferroni')
# Print the resulting conclusions
print(p_adjusted[0])

# Print the adjusted p-values themselves 
print(p_adjusted[1])
print("////////////")


{'columnTot': {'pp': 1, 'Tau': 1, 'NaN': 2, 'va': 1, 'po': 1}, 'Warning': {'pp': 1, 'NaN': 1, 'po': 1}, 'DistractClient': {'Tau': 1, 'va': 1}, 'NoResponse': {'NaN': 1}}
                Tau   po   pp   va
columnTot       1.0  1.0  1.0  1.0
Warning         0.0  1.0  1.0  0.0
DistractClient  1.0  0.0  0.0  1.0
NoResponse      0.0  0.0  0.0  0.0
Warning
Warning
skip
DistractClient
DistractClient
skip
NoResponse
NoResponse
skip


ZeroDivisionError: float division by zero

In [17]:

def chi2_cont_row(df):
    stat, p, dof, expected = chi2_contingency(df, correction=True)
    print('dof=%d' % dof)
    print(expected)

    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')


    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    return stat, p, dof, expected 

In [11]:
def calcFreqMeasures2DDict(data, cCurrT, columnResponse, cNextT, setResponses):
    F = dict()
    for index, row in data.iterrows():
        currT = row[cCurrT]
        nextT = row[cNextT]
        for cMeasureValue in setResponses:
            values = str(row[columnResponse]).split(';')
            values = [x.strip() for x in values ]
            if(cMeasureValue in values):
                if currT not in F:
                    F[currT] = dict()
                    F[currT]["columnTot"] = dict()
                if cMeasureValue not in F[currT]:
                    F[currT][cMeasureValue] = dict()
                if nextT not in F[currT][cMeasureValue]:
                    if nextT != np.nan:
                        F[currT][cMeasureValue][nextT] = 0
                F[currT][cMeasureValue][nextT] += 1      
        if nextT not in F[currT]["columnTot"]:
            F[currT]["columnTot"][nextT] = 0
        F[currT]["columnTot"][nextT] += 1
        
    return F